In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.4.8-cp37-cp37m-linux_x86_64.whl size=4606579 sha256=d9e7f4be4407bceb74dd7be7d1f0d9d479caf8fa78c0ef93a6d200e0d76ccb3f
  Stored in directory: /root/.cache/pip/wheels/88/e6/34/25e73cccbaf1a961154bb562a5f86123b68fdbf40e306073d6
Successfully built implicit


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import choice

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [4]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [6]:
test_users = result.shape[0]
new_test_users = len(set(data_test['user_id']) - set(data_train['user_id']))

print('В тестовом дата сете {} юзеров'.format(test_users))
print('В тестовом дата сете {} новых юзеров'.format(new_test_users))

В тестовом дата сете 2042 юзеров
В тестовом дата сете 0 новых юзеров


In [7]:
def weighted_random_recommendation(items_id,items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    items = np.array(items_id)
    weight = np.array(items_weights)
    weight /= weight.sum()
    recs = np.random.choice(items, size=n, p=weight, replace=False)
    
    return recs.tolist()

In [8]:
%%time

# your_code
items = data_train.item_id.unique()

weight = data_train.groupby('item_id')['sales_value'].sum().reset_index()
weight['sales_value'] = abs(np.log(weight['sales_value']))
weight['sales_value'].replace([np.inf, -np.inf], 0, inplace=True)
weight_log = weight['sales_value']


result['random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items,weight_log, n=5))

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: user 3.64 s, sys: 198 ms, total: 3.84 s
Wall time: 3.88 s


In [9]:
result.head(2)

,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9418443, 1001445, 9526885, 1437522, 84387]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1125837, 15687001, 55021, 13133638, 868818]"


## Задание 2. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая случаный на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [10]:
# your_code

popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

popularity.head()

,item_id,n_sold
0,25671,6
1,26081,1
2,26093,1
3,26190,1
4,26355,2


In [11]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [13]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[ ~ data_train['item_id'].isin(top_5000), 'item_id'] = 6666
data_train.head(100)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,1060,26985040735,1,9553288,1,8.49,315,0.00,1251,1,0.0,0.0
96,1351,26985052379,1,903230,1,0.99,447,-0.30,1955,1,0.0,0.0
97,744,26985165432,1,6666,0,0.00,31582,0.00,1119,1,0.0,0.0
98,212,26985205886,1,822346,1,1.25,288,-0.34,1341,1,0.0,0.0


In [14]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity',
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix[user_item_matrix > 0] = 1 # так как в итоге хотим предсказать 

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()



In [15]:
user_item_matrix.shape

(2499, 5001)

In [16]:
user_item_matrix.sum().sum() / (user_item_matrix.shape[0] * user_item_matrix.shape[1]) * 100

5.33770796861036

In [17]:
# создаем словари мапинга между id бизнеса к строчному id матрицы

userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [21]:
%%time

model = ItemItemRecommender(K=10, num_threads=4) # K - кол-во билжайших соседей


model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)


recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)

  0%|          | 0/5001 [00:00<?, ?it/s]

CPU times: user 2.9 s, sys: 37.7 ms, total: 2.94 s
Wall time: 2.24 s


In [22]:
recs

[(0, 78679.0),
 (3408, 72173.0),
 (2149, 60376.0),
 (3587, 46199.0),
 (301, 42769.0)]

In [23]:
[id_to_itemid[rec[0]] for rec in recs]

[6666, 1082185, 981760, 1098066, 840361]

In [26]:
%%time

result['itemitem'] = result['user_id'].apply(lambda user_id: [
                                     id_to_itemid[rec[0]]  for rec in model.recommend(userid=userid_to_id[user_id], 
                                                user_items=sparse_user_item,   # на вход user-item matrix
                                                N=5, 
                                                filter_already_liked_items=False, 
                                                filter_items=None, 
                                                recalculate_user=True)
                                                             ])

CPU times: user 89.5 ms, sys: 0 ns, total: 89.5 ms
Wall time: 93.1 ms


В начале построим со старыми данными random_recommendation

In [27]:
result.head(5)

,user_id,actual,random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[9418443, 1001445, 9526885, 1437522, 84387]","[6666, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1125837, 15687001, 55021, 13133638, 868818]","[6666, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[15572067, 1055996, 964065, 1122488, 9836695]","[6666, 1082185, 981760, 995242, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[2155788, 870386, 1217035, 13381468, 1110077]","[6666, 1082185, 981760, 995242, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[15686479, 2005115, 6904396, 932313, 391880]","[6666, 1082185, 981760, 1098066, 995242]"


In [29]:
%%time

# your_code
items = data_train.item_id.unique()

weight = data_train.groupby('item_id')['sales_value'].sum().reset_index()
weight['sales_value'] = abs(np.log(weight['sales_value']))
weight['sales_value'].replace([np.inf, -np.inf], 0, inplace=True)
weight_log = weight['sales_value']


result['random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items,weight_log, n=5))

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:726: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


CPU times: user 421 ms, sys: 1.23 ms, total: 422 ms
Wall time: 425 ms


Теперь посмотрим на то, как изменились данные если использовать только топ 5000

In [30]:
result.head(5)

,user_id,actual,random_recommendation,itemitem
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[844023, 1005456, 1015280, 7168619, 9297474]","[6666, 1082185, 981760, 995242, 840361]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[9527066, 6979753, 981086, 821200, 905235]","[6666, 1082185, 981760, 1098066, 826249]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[871279, 992128, 1093030, 955275, 845774]","[6666, 1082185, 981760, 995242, 840361]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[885750, 890739, 1077373, 983267, 847066]","[6666, 1082185, 981760, 995242, 1098066]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[866950, 8090519, 9245108, 1027090, 1002739]","[6666, 1082185, 981760, 1098066, 995242]"
